In [3]:
%run "~/ag1000g/selective_sweeps/scripts/sweep_tools.py"
samples = pd.read_csv("../../data/samples.meta.txt", sep='\t')

### Parsing the incredibly poorly written LDNe output

The output is a mixture of different lengths whitespaces, and so requires some parsing....

In [4]:
pops = samples.population.unique()
chroms = ['2L', '2R', '3L', '3R', 'X']

Ne_Ag = dict()
Ne_chrom = dict()

for pop in pops:
    for chrom in chroms:
        df = pd.read_csv(f"LDNe/Ag_LDNe_{pop}_{chrom}.out", 
                         header=None, 
                         sep='\n')
        df = df[0].str.split('\s\=\s', expand=True) #split first column using equal sign
        df.columns = ['one', 'two']
        start = np.where(df.one.str.contains('Harmonic Mean'))[0][0] #find start of results
        end = len(df)-3
        df = df.iloc[start:end]                                           #subset 
        
        #loop through rows and remove multiple whitespaces to one whitespace only, then split to new columns
        res=pd.DataFrame()
        for i in range(5):
            results = ' '.join(df.iloc[i,1].split()).split()
            res = res.append(pd.Series(results),ignore_index=True)
        
        #get columns which have estimates for CIs (poorly parsed)
        df2 = df.iloc[6:9,]
        jack = df.iloc[9,0]
        df2.one = df2.one.str.replace("*", "")
        df2 = df2.one.str.split(expand=True).reset_index(drop=True)
        
        #loop through rows and remove multiple whitespaces to one whitespace only, then split to new columns
        for i in range(len(df2)):
            df2.iloc[i,0] = ' '.join(df2.iloc[i,0].split())
        
        #extract specific estimates,works only because all files are parsed identically, not ideal
        para_1 = df2.loc[0,1:4]
        para_2 = df2.loc[1,0:3]
        jack_1 = df2.loc[2,3:7]
        jack_2 = pd.Series(jack.split())
        cols = df.iloc[:5]['one']
        cols = cols.append(pd.Series(['Parametric CI - lower', 
                              'Parametric CI - upper', 
                              'Jackknife CI - lower', 
                              'Jackknife CI - upper']), ignore_index=True)
        
        #append the  estimates to the results 
        df = res.append(para_1.reset_index(drop=True)).append(para_2.reset_index(drop=True)).append(jack_1.reset_index(drop=True)).append(jack_2, ignore_index=True)
        
        #join results to the parameter column and change column names 
        final_results = pd.concat([cols,df], axis=1)
        final_results.columns = ['Parameter', 
                         'minAF_0.05', 
                         'minAF_0.02', 
                         'minAF_0.01', 
                         'minAF_0+']
        
        #store results in dict and then save within nested dict
        Ne_chrom[chrom] = final_results
        
    Ne_Ag[pop] = dict(Ne_chrom)

/home/sanj/apps/anaconda3/envs/pysanj/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [5]:
Ne = pd.DataFrame()

# transpose, add population and chromosome columns, then re-order columns, and combine all tables for each
# pop into one big one
for pop in pops:
    for chrom in chroms:
        
        Ne_Ag[pop][chrom] = Ne_Ag[pop][chrom].set_index('Parameter').T
        
        Ne_Ag[pop][chrom]['pop'] = pop
        Ne_Ag[pop][chrom]['chrom'] = chrom

                # get a list of columns
        cols = list(Ne_Ag[pop][chrom])
        # move the column to head of list using index, pop and insert
        cols.insert(0, cols.pop(cols.index('pop')))
        Ne_Ag[pop][chrom] = Ne_Ag[pop][chrom].loc[:, cols]
        cols = list(Ne_Ag[pop][chrom])
        cols.insert(0, cols.pop(cols.index('chrom')))
        Ne_Ag[pop][chrom] = Ne_Ag[pop][chrom].loc[:, cols]
        Ne_Ag[pop][chrom]
        
        Ne = pd.concat([Ne, Ne_Ag[pop][chrom]])

In [6]:
#change names removing whitespace
Ne.columns = ['chrom', 'pop', 'sample_size', 'independent_comparisons', 
                      'overall_r^2', 'expected_r^2', 'Ne_estimate', 'Parametric_CI_lower',
                     'Parametric_CI_upper', 'Jackknife_CI_lower', 'Jackknife_CI_upper']

In [13]:
pd.options.display.max_rows = 500
Ne[(Ne.sample_size.astype(float) > 10)].sort_values(by=['pop','chrom'])

,chrom,pop,sample_size,independent_comparisons,overall_r^2,expected_r^2,Ne_estimate,Parametric_CI_lower,Parametric_CI_upper,Jackknife_CI_lower,Jackknife_CI_upper
minAF_0.05,2L,AOcol,78.0,52864480,0.014740,0.013345,236.9,235.9,237.8,139.2,642.3
minAF_0.05,2R,AOcol,78.0,61587978,0.014536,0.013345,277.7,276.5,278.9,151.2,1111.6
minAF_0.05,3L,AOcol,78.0,60285152,0.014676,0.013345,248.3,247.3,249.2,133.8,1018.3
minAF_0.05,3R,AOcol,78.0,61365987,0.014417,0.013345,308.8,307.3,310.3,169.4,1216.5
minAF_0.05,X,AOcol,78.0,54794397,0.014942,0.013345,206.7,205.9,207.4,124.0,511.1
minAF_0.05,2L,BFcol,75.0,10394007,0.013931,0.013900,10967.8,7865.0,18098.2,4321.9,Infinite
minAF_0.05,2R,BFcol,75.0,16718042,0.015388,0.013900,222.0,220.4,223.5,177.0,293.6
minAF_0.05,3L,BFcol,75.0,13145006,0.013903,0.013900,145016.7,25771.2,Infinite,15645.8,Infinite
minAF_0.05,3R,BFcol,75.0,13237401,0.013905,0.013900,72289.5,21912.8,Infinite,13362.9,Infinite
minAF_0.05,X,BFcol,75.0,8316618,0.014563,0.013900,500.7,490.3,511.5,292.0,1553.7


### Write to txt file

In [8]:
Ne.to_csv("Ne_analyses.LDNe", sep="\t", header=True, index=True, index_label='AF')

In [105]:
Ne

,chrom,pop,sample_size,independent_comparisons,overall_r^2,expected_r^2,Ne_estimate,Parametric_CI_lower,Parametric_CI_upper,Jackknife_CI_lower,Jackknife_CI_upper
minAF_0.05,3L,GHcol,55.0,6465956,0.019633,0.019242,851.2,806.9,900.7,338.5,Infinite
minAF_0.02,3L,GHcol,55.0,16165678,0.019660,0.019241,793.2,768.2,819.7,301.2,Infinite
minAF_0.01,3L,GHcol,55.0,26165685,0.019652,0.019241,808.2,787.7,829.7,298.0,Infinite
minAF_0+,3L,GHcol,55.0,55799751,0.019358,0.019240,2824.7,2662.3,3008.0,659.3,Infinite
minAF_0.05,3R,GHcol,55.0,6705702,0.019729,0.019240,680.4,652.1,711.2,224.3,Infinite
minAF_0.02,3R,GHcol,55.0,16994552,0.019748,0.019241,655.6,638.8,673.2,206.5,Infinite
minAF_0.01,3R,GHcol,55.0,26572215,0.019739,0.019241,667.2,653.2,681.7,212.0,Infinite
minAF_0+,3R,GHcol,55.0,56095464,0.019436,0.019241,1702.3,1641.8,1767.3,409.9,Infinite
minAF_0.05,3L,GHgam,12.0,13844967,0.109337,0.108240,279.1,259.7,301.6,98.9,Infinite
minAF_0.02,3L,GHgam,12.0,50793262,0.104740,0.108237,Infinite,Infinite,Infinite,Infinite,Infinite


## IF JUST one AF is run! 

In [35]:
final_results.iloc[:,:2]

,Parameter,minAF_0.05
0,Harmonic Mean Sample Size,78.0
1,Independent Comparisons,365890559
2,OverAll r^2,0.014644
3,Expected r^2 Sample,0.013345
4,Estimated Ne^,254.4
5,Parametric CI - lower,254.0
6,Parametric CI - upper,254.8
7,Jackknife CI - lower,138.3
8,Jackknife CI - upper,1002.7
